# wetterdienst - A basic notebook example

In [1]:
#!pip install wetterdienst

## Import modules necessary for general functioning

In [2]:
from pprint import pprint

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import polars as pl
from matplotlib import colormaps

from wetterdienst import Settings
from wetterdienst.provider.dwd.observation import (
    DwdObservationDataset,
    DwdObservationParameter,
    DwdObservationPeriod,
    DwdObservationRequest,
    DwdObservationResolution,
)

In [3]:
%matplotlib inline

Wetterdienst default settings

In [4]:
# don't display real cache_dir, which might contain sensible information
settings = Settings.default().to_dict()
settings.update({"cache_dir": "abc"})
settings

## Which parameters are available?

All available parameters

In [5]:
pprint(DwdObservationRequest.discover())

Selection of daily data

In [6]:
pprint(DwdObservationRequest.discover(resolution=DwdObservationResolution.DAILY))

## List of historical daily precipitation stations of DWD (Germany)

In [7]:
request = DwdObservationRequest(
    parameters=DwdObservationDataset.PRECIPITATION_MORE,
    resolution=DwdObservationResolution.DAILY,
    period=DwdObservationPeriod.HISTORICAL,
).all()
request.df.head()

Number of stations with available data

In [8]:
request.df.shape

## Map of historical daily precipitation stations of DWD (Germany)

In [9]:
cmap = colormaps.get_cmap("viridis")
bounds = np.quantile(request.df.get_column("height"), [0, 0.25, 0.5, 0.75, 1])
norm = mpl.colors.BoundaryNorm(bounds, cmap.N)
fig, ax = plt.subplots(figsize=(10, 10), tight_layout=True)
plot = request.df.to_pandas().plot.scatter(x="longitude", y="latitude", c="height", cmap=cmap, norm=norm, ax=ax)
title = "Map of daily precipitation stations in Germany"
plot.set_title(title)
plt.show()

## Get data for daily precipitation station - Dresden Klotzsche (1048)

In [10]:
values = (
    DwdObservationRequest(
        parameters=DwdObservationDataset.PRECIPITATION_MORE,
        resolution=DwdObservationResolution.DAILY,
        period=DwdObservationPeriod.HISTORICAL,
    )
    .filter_by_station_id(station_id=[1048])
    .values.all()
    .df
)
values.drop_nulls().head()

## Get data for daily precipitation and temperature (tmin, tmean, tmax) for station - Dresden Klotzsche (1048)

In [11]:
parameters = [
    DwdObservationParameter.DAILY.TEMPERATURE_AIR_MEAN_2M,
    DwdObservationParameter.DAILY.TEMPERATURE_AIR_MAX_2M,
    DwdObservationParameter.DAILY.TEMPERATURE_AIR_MIN_2M,
    DwdObservationParameter.DAILY.PRECIPITATION_HEIGHT,
]
values = (
    DwdObservationRequest(
        parameters=parameters,
        resolution=DwdObservationResolution.DAILY,
        period=DwdObservationPeriod.HISTORICAL,
    )
    .filter_by_station_id(station_id=(1048,))
    .values.all()
)
values.df.drop_nulls().head()

## Aggregate to annual values

In [12]:
data = []
for (parameter,), group in values.df.groupby([pl.col("parameter")], maintain_order=True):
    if parameter == "precipitation_height":
        agg_df = group.groupby(pl.col("date").dt.year(), maintain_order=True).agg(
            pl.when(pl.col("value").is_not_null().sum() > 330).then(pl.col("value").sum()),
        )
    else:
        agg_df = group.groupby(pl.col("date").dt.year(), maintain_order=True).agg([pl.col("value").mean()])
    agg_df = agg_df.select(
        pl.lit(parameter).alias("parameter"),
        pl.col("date").cast(pl.Utf8).str.to_datetime("%Y"),
        pl.col("value"),
    )
    data.append(agg_df)
df_annual = pl.concat(data)
df_annual

## Create some plots for the data

In [13]:
cmap = plt.get_cmap("viridis", 4)
colors = cmap.colors
fig, axes = plt.subplots(nrows=len(parameters), tight_layout=True, sharex=True, figsize=(10, 10))  #
for (parameter, daily), (_, annual), ax, color in zip(
    values.df.groupby("parameter", maintain_order=True),
    df_annual.groupby("parameter", maintain_order=True),
    axes,
    colors,
):
    if parameter == "precipitation_height":
        ax2 = ax.twinx()
    else:
        ax2 = ax
    daily.to_pandas().plot(x="date", y="value", label=parameter, alpha=0.75, ax=ax, c=color, legend=False)
    annual.to_pandas().plot(
        x="date",
        y="value",
        kind="line",
        label=f"annual({parameter})",
        alpha=0.75,
        ax=ax2,
        c="black",
        legend=False,
    )
    ax.legend(loc=0)
    if ax != ax2:
        ax2.legend(loc=3)
plt.suptitle("Temperature and precipitation time series of Dresden, Germany", y=1.01)
plt.show()

## Find a station

In [14]:
request = DwdObservationRequest(
    parameters=DwdObservationParameter.DAILY.CLIMATE_SUMMARY.TEMPERATURE_AIR_MEAN_2M,
    resolution=DwdObservationResolution.DAILY,
    period=DwdObservationPeriod.HISTORICAL,
    start_date="1930-01-01",
    end_date="1970-01-01",
)

In [15]:
request.filter_by_rank((51.05089, 13.73832), 5).df

In [16]:
request.filter_by_distance((51.05089, 13.73832), 20).df

In [17]:
request.filter_by_station_id("1048").df

In [18]:
request.filter_by_name("Dresden Klo", threshold=70).df

In [19]:
request.filter_by_sql("SELECT * FROM df WHERE name LIKE 'Dresden%' AND ABS(height - 200) < 30").df

In [20]:
request.filter_by_bbox(left=13.74, bottom=51.126, right=13.755, top=51.13).df

## Summarize values
Fill up values from nearby stations

In [21]:
request.summarize((51.05089, 13.73832)).df

## Interpolate values

In [22]:
request.interpolate((51.05089, 13.73832)).df